In [11]:
%run feature_generation.py

(42, 12)
(42, 12)
(35, 12)
(39, 12)
(70, 12)
(25, 12)
(34, 12)
(36, 12)
(35, 12)
(9, 12)
(18, 12)
(54, 12)
(28, 12)
(53, 12)
(62, 12)
(64, 12)
(67, 12)
(26, 12)
(20, 12)
(40, 12)
(13, 12)
(16, 12)
(103, 12)
(118, 12)
(7, 12)
(27, 12)
(110, 12)
(33, 12)
(18, 12)
(22, 12)
(22, 12)
(20, 12)
(18, 12)
(17, 12)
(18, 12)
(20, 12)
(19, 12)
(19, 12)
(31, 12)
(30, 12)
(28, 12)
(30, 12)
(29, 12)
(28, 12)
(30, 12)
(29, 12)
(27, 12)
(33, 12)
(29, 12)
(25, 12)
(26, 12)
(31, 12)
(66, 12)
(68, 12)
(62, 12)
(33, 12)
(30, 12)
(96, 12)
(32, 12)
(22, 12)
(15, 12)
(18, 12)
(10, 12)
(43, 12)
(43, 12)
(40, 12)
(23, 12)
(22, 12)
(21, 12)
(26, 12)
(27, 12)
(26, 12)
(27, 12)
(26, 12)
(44, 12)
(29, 12)
(26, 12)
(38, 12)
(28, 12)
(31, 12)
(29, 12)
(37, 12)
(36, 12)
(32, 12)
(32, 12)
(39, 12)
(29, 12)
(11, 12)
(9, 12)
(15, 12)
(15, 12)
(54, 12)
(13, 12)
(30, 12)
(30, 12)
(19, 12)
(18, 12)
(31, 12)
(20, 12)
(24, 12)
(20, 12)
(22, 12)
(23, 12)
(18, 12)
(19, 12)
(27, 12)
(21, 12)
(23, 12)
(32, 12)
(11, 12)
(47, 12)
(

In [24]:
# %load "Data preprocessing.py"
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_selection import VarianceThreshold


atom_features = pd.read_csv('atom_features_PCA.csv')

smiles = atom_features['SMILES']

bond_features = pd.read_csv('bond_features_PCA.csv')

#seed(s)
# smilesF = pd.read_csv('SMILES_200Features.csv')
# del smilesF['SMILES']

# sheader = []

# smilesF = smilesF.fillna(0)

#smilesF2 = smilesF.transpose()
# sheader = list(smilesF.columns.values)
# scaler = preprocessing.StandardScaler()
# scaled = scaler.fit_transform(smilesF)
# scaled_df = pd.DataFrame(scaled, columns = sheader)
#scaled_df = scaled_df.transpose()



# del atom_features['compound_stem']
del atom_features['SMILES']

atom_features = atom_features.add_prefix('A_')

# del bond_features['compound_stem']
# del bond_features['SMILES']

bond_features = bond_features.add_prefix('B_')

for i in range (0,12):
  i = str(i)
  column = 'B_'+i
  b1 = bond_features[column]
  
  name = column
  i = int(i)
  atom_features.insert(i,name,b1)

# header_smilesf = []
# header_smilesf = list(smilesF.columns.values)

#tf.random.set_seed(s)
#seed(s)
second_pca = PCA(n_components = 50) 
data_atom_pca = second_pca.fit_transform(atom_features)

pcaNames = []
for p in range(1,51):
  pc = str(p)
  pca = 'PCA'+pc
  pcaNames.append(pca)


data_atom_pca = pd.DataFrame(data=data_atom_pca, columns=pcaNames)

j = 0
for col in pcaNames:
  col_data = data_atom_pca[col]
  # scaled_df.insert(j,col,col_data)
  
  j = j+1


sel = VarianceThreshold(0)
cleaned = sel.fit_transform(atom_features)


In [26]:
# %load "VAE & K-Means model&train.py"
import numpy as np
import pandas as pd
import torch
import tensorflow as tf
from numpy.random import seed 
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from scipy import stats
from scipy.stats import norm
import seaborn as sns
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import random
from keras.models import load_model
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device
data_set = np.float32(cleaned)
trainloader=DataLoader(dataset=data_set,batch_size=1024)

class Autoencoder(nn.Module):
    def __init__(self,D_in,H=50,H2=12,latent_dim=32):
        
        #Encoder
        super(Autoencoder,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H2)
        
#         # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

#         # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)
        
#         # Decoder
        self.linear4=nn.Linear(H2,H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5=nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6=nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)
        
        self.relu = nn.ReLU()
        
    def encode(self, x):
        lin1 = self.relu(self.lin_bn1(self.linear1(x)))
        lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))
        lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc1 = F.relu(self.bn1(self.fc1(lin3)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)
        
        return r1, r2
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def decode(self, z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        fc4 = self.relu(self.fc_bn4(self.fc4(fc3)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc4)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))


        
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        
        return self.decode(z), mu, logvar
###################################################################
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")
    
    # 
    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD
######################################################################
# takes in a module and applies the specified weight initialization
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    # for every Linear layer in a model..
    if classname.find('Linear') != -1:
        # get the number of the inputs
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.fill_(0)
########################################################################
D_in = 243
H = 50
H2 = 12
model = Autoencoder(D_in, H, H2).to(device)
model.apply(weights_init_uniform_rule)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_mse = customLoss()

#########################################################################
# Training
epochs = 1000
log_interval = 50
val_losses = []
train_losses = []

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
#        if batch_idx % log_interval == 0:
#            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                epoch, batch_idx * len(data), len(trainloader.dataset),
#                       100. * batch_idx / len(trainloader),
#                       loss.item() / len(data)))
    if epoch % 200 == 0:        
        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))

for epoch in range(1, epochs + 1):
    train(epoch)

###########################################################################
mu_output = []
logvar_output = []

with torch.no_grad():
    for i, (data) in enumerate(trainloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)

            
            mu_tensor = mu   
            mu_output.append(mu_tensor)
            mu_result = torch.cat(mu_output, dim=0)

            logvar_tensor = logvar   
            logvar_output.append(logvar_tensor)
            logvar_result = torch.cat(logvar_output, dim=0)
##################################################################################

pred = mu_result.cpu().detach().numpy() 
pred.shape
np.save('pred_vae32_1500_epochs.npy', pred)

####################################################################################





RuntimeError: mat1 and mat2 shapes cannot be multiplied (1024x157 and 243x50)